In [2]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
import pandas as pd
import numpy as np
warnings.filterwarnings('ignore')

# 1. 데이터 로드
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
submission = pd.read_csv("../data/sample_submission.csv")

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"Submission shape: {submission.shape}")

# 2. 전처리 함수 (train, test 동일하게 적용)
def preprocess(df):
    df_copy = df.copy()
    
    # 시술_대분류
    def major_procedure(x):
        if pd.isna(x):
            return "Unknown"
        if "IUI" in x:
            return "IUI"
        if "DI" in x:
            return "Other"
        if "ICSI" in x:
            return "ICSI"
        if "IVF" in x:
            return "IVF"
        return "Other"
    
    df_copy["시술_대분류"] = df_copy["특정 시술 유형"].apply(major_procedure)
    
    # BLASTOCYST 포함 여부
    df_copy["BLASTOCYST_포함"] = df_copy["특정 시술 유형"].str.contains("BLASTOCYST", na=False).astype(int)
    
    # 배아 이식 여부 (올바른 버전 - 21개 컬럼 전부 체크)
    embryo_stage_cols = [
        "단일 배아 이식 여부",
        "착상 전 유전 진단 사용 여부",
        "배아 생성 주요 이유",
        "총 생성 배아 수",
        "미세주입된 난자 수",
        "미세주입에서 생성된 배아 수",
        "이식된 배아 수",
        "미세주입 배아 이식 수",
        "저장된 배아 수",
        "미세주입 후 저장된 배아 수",
        "해동된 배아 수",
        "해동 난자 수",
        "수집된 신선 난자 수",
        "저장된 신선 난자 수",
        "혼합된 난자 수",
        "파트너 정자와 혼합된 난자 수",
        "기증자 정자와 혼합된 난자 수",
        "동결 배아 사용 여부",
        "신선 배아 사용 여부",
        "기증 배아 사용 여부",
        "대리모 여부",
    ]
    
    df_copy["배아_이식_미도달"] = df_copy[embryo_stage_cols].isna().all(axis=1).astype(int)
    df_copy["배아_이식_여부"] = 1 - df_copy["배아_이식_미도달"]
    
    # 총시술_bin3
    def collapse_trials(x):
        if x == '0회':
            return '0회'
        elif x in ['1회', '2회']:
            return '1–2회'
        else:
            return '3회 이상'
    
    df_copy["총시술_bin3"] = df_copy["총 시술 횟수"].apply(collapse_trials)
    
    # 나이_3구간
    def age_group_simple(age):
        if age == '알 수 없음':
            return 'Unknown'
        elif age == '만18-34세':
            return '34세 이하'
        elif age in ['만35-37세', '만38-39세']:
            return '35-39세'
        else:
            return '40세 이상'
    
    df_copy['나이_3구간'] = df_copy['시술 당시 나이'].apply(age_group_simple)
    
    # 이식배아_구간
    def embryo_count_bin(count):
        if pd.isna(count) or count == 0:
            return '0개'
        elif count <= 2:
            return '1-2개'
        else:
            return '3개 이상'
    
    df_copy['이식배아_구간'] = df_copy['이식된 배아 수'].apply(embryo_count_bin)
    
    # Day5_이식_여부
    df_copy['Day5_이식_여부'] = (df_copy['배아 이식 경과일'] == 5.0).astype(int)
    
    # 불임원인_복잡도
    infertility_cols = [
        "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
        "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인",
        "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
        "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
        "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태"
    ]
    
    df_copy["불임_원인_개수"] = df_copy[infertility_cols].sum(axis=1)
    
    def infertility_complexity(count):
        if count == 0:
            return 'None'
        elif count == 1:
            return 'Single'
        elif count == 2:
            return 'Double'
        else:
            return 'Multiple'
    
    df_copy['불임원인_복잡도'] = df_copy['불임_원인_개수'].apply(infertility_complexity)
    
    # 배아_해동_실시_여부
    df_copy['배아_해동_실시_여부'] = df_copy['배아 해동 경과일'].notna().astype(int)
    
    return df_copy

# 3. 전처리 적용
train_df = preprocess(train_df)
test_df = preprocess(test_df)

# 배아_이식_여부 확인
print(f"\n배아_이식_여부 분포 (Train):")
print(train_df['배아_이식_여부'].value_counts())

# 4. Feature 준비
feature_cols = [col for col in train_df.columns if col not in ['ID', '임신 성공 여부']]

X_train = train_df[feature_cols].copy()
y_train = train_df['임신 성공 여부'].copy()
X_test = test_df[feature_cols].copy()

# 5. 범주형 변수 지정
categorical_features = [
    '시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형', '배란 유도 유형',
    '배아 생성 주요 이유', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수',
    'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수',
    'IVF 출산 횟수', 'DI 출산 횟수', '난자 출처', '정자 출처', '난자 기증자 나이',
    '정자 기증자 나이', '시술_대분류', '총시술_bin3', '나이_3구간', '이식배아_구간',
    '불임원인_복잡도',
]

for col in categorical_features:
    if col in X_train.columns:
        X_train[col] = X_train[col].astype('category')
        X_test[col] = X_test[col].astype('category')

# 6. StratifiedKFold로 Cross Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'random_state': 42,
}

oof_preds = np.zeros(len(X_train))
test_preds = np.zeros(len(X_test))

print("\n=== Cross Validation ===")
for fold, (train_idx, valid_idx) in enumerate(skf.split(X_train, y_train), 1):
    print(f"\n--- Fold {fold} ---")
    
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    train_data = lgb.Dataset(X_tr, label=y_tr, categorical_feature=categorical_features)
    valid_data = lgb.Dataset(X_val, label=y_val, categorical_feature=categorical_features, reference=train_data)
    
    model = lgb.train(
        params,
        train_data,
        num_boost_round=1000,
        valid_sets=[train_data, valid_data],
        valid_names=['train', 'valid'],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=100)
        ]
    )
    
    # OOF 예측
    oof_preds[valid_idx] = model.predict(X_val, num_iteration=model.best_iteration)
    
    # Test 예측 (평균)
    test_preds += model.predict(X_test, num_iteration=model.best_iteration) / skf.n_splits
    
    fold_auc = roc_auc_score(y_val, oof_preds[valid_idx])
    print(f"Fold {fold} AUC: {fold_auc:.6f}")

# 7. 최종 CV 점수
cv_auc = roc_auc_score(y_train, oof_preds)
print(f"\n=== Final CV AUC: {cv_auc:.6f} ===")

# 8. Submission 생성
submission['probability'] = test_preds
submission.to_csv("../outputs/submission_lgbm_v2(0205).csv", index=False)
print("\n✅ Submission saved!")
print(f"Prediction stats:")
print(f"Min: {test_preds.min():.6f}")
print(f"Max: {test_preds.max():.6f}")
print(f"Mean: {test_preds.mean():.6f}")

# 9. Feature Importance (마지막 fold 모델 기준)
importance_df = pd.DataFrame({
    'feature': model.feature_name(),
    'importance': model.feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False)

print("\n=== Top 30 Feature Importance ===")
print(importance_df.head(30))

# 새로 만든 변수들의 중요도 확인
new_features = [
    '시술_대분류', 'BLASTOCYST_포함', '배아_이식_여부', '총시술_bin3',
    '나이_3구간', '이식배아_구간', 'Day5_이식_여부', '배아_해동_실시_여부', '불임원인_복잡도'
]

print("\n=== 새로 만든 변수 중요도 ===")
new_var_importance = importance_df[importance_df['feature'].isin(new_features)].sort_values('importance', ascending=False)
print(new_var_importance)

Train shape: (256351, 69)
Test shape: (90067, 68)
Submission shape: (90067, 2)

배아_이식_여부 분포 (Train):
배아_이식_여부
1    250060
0      6291
Name: count, dtype: int64

=== Cross Validation ===

--- Fold 1 ---
Training until validation scores don't improve for 50 rounds
[100]	train's auc: 0.747284	valid's auc: 0.737408
Early stopping, best iteration is:
[138]	train's auc: 0.750172	valid's auc: 0.737706
Fold 1 AUC: 0.737706

--- Fold 2 ---
Training until validation scores don't improve for 50 rounds
[100]	train's auc: 0.745694	valid's auc: 0.742068
[200]	train's auc: 0.752956	valid's auc: 0.74284
Early stopping, best iteration is:
[170]	train's auc: 0.751045	valid's auc: 0.74289
Fold 2 AUC: 0.742890

--- Fold 3 ---
Training until validation scores don't improve for 50 rounds
[100]	train's auc: 0.74649	valid's auc: 0.7397
[200]	train's auc: 0.753598	valid's auc: 0.740201
Early stopping, best iteration is:
[203]	train's auc: 0.753841	valid's auc: 0.740212
Fold 3 AUC: 0.740212

--- Fold 4 ---
Trai

In [3]:
# 중복 확인
print("=== 배아_이식_여부 vs 이식된_배아_수 ===")
print(pd.crosstab(
    train_df['배아_이식_여부'], 
    train_df['이식된 배아 수'].isna()
))

print("\n=== 배아_이식_여부 vs 이식배아_구간 ===")
print(pd.crosstab(
    train_df['배아_이식_여부'], 
    train_df['이식배아_구간']
))

=== 배아_이식_여부 vs 이식된_배아_수 ===
이식된 배아 수   False  True 
배아_이식_여부               
0              0   6291
1         250060      0

=== 배아_이식_여부 vs 이식배아_구간 ===
이식배아_구간      0개    1-2개  3개 이상
배아_이식_여부                      
0          6291       0      0
1         36544  204636   8880


In [ ]:
# 배아 진행 단계 변수 생성 - 배아_이식_여부랑 배아 이식 수가 좀 겹치는 것 같음
def embryo_stage(row):
    if row['배아_이식_여부'] == 0:
        return '배아단계_미도달'
    elif pd.isna(row['총 생성 배아 수']) or row['총 생성 배아 수'] == 0:
        return '배아생성_실패'
    elif pd.isna(row['이식된 배아 수']) or row['이식된 배아 수'] == 0:
        return '이식_미실시'
    else:
        return '이식_완료'

train_df['배아_진행_단계'] = train_df.apply(embryo_stage, axis=1)

# 단계별 임신 성공 확률 확인
print("=== 배아_진행_단계별 임신 성공률 ===")
stage_stats = train_df.groupby('배아_진행_단계')['임신 성공 여부'].agg([
    ('count', 'count'),
    ('success_rate', 'mean')
]).sort_values('success_rate', ascending=False)

print(stage_stats)
print(f"\n전체 평균 성공률: {train_df['임신 성공 여부'].mean():.4f}")

=== 배아_진행_단계별 임신 성공률 ===
           count  success_rate
배아_진행_단계                      
이식_완료     176466      0.318617
배아생성_실패    53349      0.171868
배아단계_미도달    6291      0.128914
이식_미실시     20245      0.001136

전체 평균 성공률: 0.2583
